<a href="https://colab.research.google.com/github/RodrigoEche/00_CoderHouseProyecto/blob/main/Entrega_Complementaria_Cross_Validation_y_Mejora_de_modelos_Rodrigo_Echegoyemberry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***DESAFÍO 15 CROSS VALIDATION Y MEJORA DE MODELOS***

In [ ]:
#  -------------------------------------------------------------------------------------------------------------------------------
# | CONSIGNAS:                                                                                                                     |
# |       1. USAR TRABAJO PREVIO (PARTE 1)
# |       2. EVALUAR LOS MODELOS CON DISTINTAS TECNICAS DE CROSS VALIDATION (PARTE 2)                                                           |
# |       3. APRENDER VENTAJA DE LA VALIDACION AUTOMATICA VERSUS  LA MANUAL QUE SE VENIA USANDO
# |       4. APRENDER EN PARTICULAR LA K-FOLD CROSS VALIDATION
# |       5. EVALUAR LOS MODELOS EN CADA CASO                                                                          |
# |       6. SACAR CONCLUSIONES
# |
# |       Consigna:  Video clase 49 en 1h 48 min                                                                                                               |                                                                                     |
# |       Ejemplo:   https://www.kaggle.com/code/satishgunjal/tutorial-k-fold-cross-validation                                                                                                                      |
#  -------------------------------------------------------------------------------------------------------------------------------

#Cargo bibliotecas

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

#Cargo dataset

In [2]:
archivo = 'https://raw.githubusercontent.com/RodrigoEche/DS_Datasets/main/1%20Sales%20Data%20for%20Economic%20Data%20Analysis/Sales%20Data%20for%20Economic%20Data%20Analysis%20Kaggle/salesforcourse-4fe2kehu.csv'
dataset= pd.read_csv( archivo, sep=',')
dataset.shape

(34867, 16)

In [3]:
prueba = dataset[['Unit Cost','Unit Price']]
prueba

,Unit Cost,Unit Price
0,80.00,109.000000
1,24.50,28.500000
2,3.67,5.000000
3,87.50,116.500000
4,35.00,41.666667
...,...,...
34862,1160.00,985.500000
34863,2049.00,1583.000000
34864,683.00,560.666667
34865,2320.00,1568.000000


In [4]:
list(dataset.columns)

['index',
 'Date',
 'Year',
 'Month',
 'Customer Age',
 'Customer Gender',
 'Country',
 'State',
 'Product Category',
 'Sub Category',
 'Quantity',
 'Unit Cost',
 'Unit Price',
 'Cost',
 'Revenue',
 'Column1']

#Diccionario

######El dataset corresponde a venta de artículos de ciclismo por internet a diferentes países

Feature     | Descripción
------------|------------
'index', |  indice, no es útil
 'Date',| fecha, util para graficar pero no es relevante para modelar
 'Year',|año, util para el EDA.
 'Month',|mes, idem anterior
 'Customer Age',| edad del cliente
 'Customer Gender',|género del cliente
 'Country',|país desde donde se compró
 'State',|estado
 'Product Category',|categoría del producto
 'Sub Category',| subcategoría
 'Quantity',| cantidad de artículos comprado en la operación
 'Unit Cost',|costo de cada artículo
 'Unit Price',| precio de venta de cada artículo
 'Cost',|costos totales
 'Revenue',|ganancias
 'Column1'|columna vacía

#**PARTE 1: MODELADO CON VALIDACIÓN SIMPLE**

#Elegir modelo de regresión simple o múltiple

In [5]:
# el mismo código se usa en regresion simple o múltiple, por eso se debe elegir
regresionSimple = 0

#Feature engineering

######A diferencia del modelo simple en que solo hay dos columnas la predictora y la objetivo, en la regresion múltiple se usan muchas predictoras por eso requiere un mayor procesamiento a través de la Feature engineering

In [6]:
df = dataset.copy()

# Borro ultimo renglon con solo NaN
df = df.drop(df.index[-1],  axis=0) #elimino ultimo renglon basura lleno de NaN

# Acondicionado fechas para usar en gráficas de análisis EDA
df['Date'] = pd.to_datetime(df['Date'])

# Elimino columnas que no sirven al modelo
df= df.drop(columns = ['Column1','Date','index'],axis=1)

# Si es regresión simple me quedo solo con dos variables: la explicativa Cost y la objetivo Price
if regresionSimple == 1: df= df[['Unit Cost','Unit Price']]

print(df.shape)
print(df.columns)

(34866, 13)
Index(['Year', 'Month', 'Customer Age', 'Customer Gender', 'Country', 'State',
       'Product Category', 'Sub Category', 'Quantity', 'Unit Cost',
       'Unit Price', 'Cost', 'Revenue'],
      dtype='object')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34866 entries, 0 to 34865
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              34866 non-null  float64
 1   Month             34866 non-null  object 
 2   Customer Age      34866 non-null  float64
 3   Customer Gender   34866 non-null  object 
 4   Country           34866 non-null  object 
 5   State             34866 non-null  object 
 6   Product Category  34866 non-null  object 
 7   Sub Category      34866 non-null  object 
 8   Quantity          34866 non-null  float64
 9   Unit Cost         34866 non-null  float64
 10  Unit Price        34866 non-null  float64
 11  Cost              34866 non-null  float64
 12  Revenue           34866 non-null  float64
dtypes: float64(7), object(6)
memory usage: 3.5+ MB


In [8]:
# Codifico las columnas categóricas como números
data = df.copy()
categoricas = data.select_dtypes(include=['object'])
numericas   = data.select_dtypes(exclude=['object'])

# Codifico con get_dummies() las variables categóricas
if regresionSimple == 1:
  categoricas = pd.DataFrame()
else:
  categoricas = pd.get_dummies(categoricas)

In [9]:
# Concateno dataframes de categoricas codificadas y numericas
data = pd.concat([numericas, categoricas], axis=1)

# Obtengo el df para el modelo luego del data wrangling
print(data.shape)
#print(data.columns)
data.sample(2)

(34866, 90)


,Year,Customer Age,Quantity,Unit Cost,Unit Price,Cost,Revenue,Month_April,Month_August,Month_December,...,Sub Category_Helmets,Sub Category_Hydration Packs,Sub Category_Jerseys,Sub Category_Mountain Bikes,Sub Category_Road Bikes,Sub Category_Shorts,Sub Category_Socks,Sub Category_Tires and Tubes,Sub Category_Touring Bikes,Sub Category_Vests
10992,2015.0,33.0,2.0,60.0,67.0,120.0,134.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8258,2015.0,38.0,1.0,57.0,67.0,57.0,67.0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


# Modelado del problema de negocio

### El problema de negocio por tener el Precio como objetivo a predecir es un modelo supervisado, con el Precio como variable objetivo, target o label y con variables predictoras las restantes por lo que asi planteado es una Regresion múltiple o simple en caso de elegir solo una variable explicativa.


## Regresión lineal simple (saltear celda si se eligió regresión múltiple)

In [ ]:
# X: variables independientes, explicativas o predictoras.
# y: variable  dependiente, objetivo o target.
Xsimple = data['Unit Cost']   #.drop(columns = ['Unit Price'])
ysimple = data['Unit Price']

In [ ]:
# Separo el conjunto en datos para Train y para Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xsimple, ysimple, test_size=0.33, random_state=42)

# Elijo, selecciono, creo o instancio el modelo de regresión lineal
simple = LinearRegression()

# Ajusto o entreno el modelo
# mostrándole y_train correspondiente a cada X_train
simple.fit(X_train, y_train)

# Calculo predicciones con el modelo ya entrenado y con las X de test
y_pred = simple.predict(X_test)

# Evalúo el modelo comparando su predicción "y_pred" con lo que debería ser "y_test"
# uso dos métricas aplicables en regresión: error cuadrático medio y R^2
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
_score = simple.score(Xsimple,ysimple)

# Imprimo
print(f'Error Cuadrático Medio (MSE): {mse.round(3)}')
print(f'Coeficiente de Determinación (R^2): {r2.round(3)}')
print(f'Score del modelo: {_score.round(3)}')
print(f'Modelo con {Xsimple.shape[1]} feature.')

## Regresión lineal múltiple

In [11]:
# X: variables independientes, explicativas o predictoras.
# y: variable  dependiente, objetivo o target.
X = data.drop(columns = ['Unit Price'])   # todas menos 'Unit Price'
y = data['Unit Price']                    # Variable a predecir

## En validación simple separo en un lote grande de entrenamiento y un lote más chico de validación

In [13]:
# Separo el conjunto en datos para Train y para Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Elijo, selecciono, creo o instancio el modelo de regresión lineal
modelo = LinearRegression()

# Ajusto o entreno el modelo
# mostrándole y_train correspondiente a cada X_train
modelo.fit(X_train, y_train)

# Calculo predicciones con el modelo ya entrenado y con las X de test
y_pred = modelo.predict(X_test)

# Evalúo el modelo comparando su predicción "y_pred" con lo que debería ser "y_test"
# uso dos métricas aplicables en regresión: error cuadrático medio y R^2
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
_score = modelo.score(X,y)

# Imprimo
print(f'Error Cuadrático Medio (MSE): {mse.round(3)}')
print(f'Coeficiente de Determinación (R^2): {r2.round(3)}')
print(f'Score del modelo: {_score.round(3)}')
print(f'Modelo con {X_train.shape[1]} features.')

Error Cuadrático Medio (MSE): 1785.893
Coeficiente de Determinación (R^2): 0.994
Score del modelo: 0.993
Modelo con 89 features.


######Las métricas de la regresión simple donde uso una sola variable explicativa dió: MSE = 10082.17, R2 = 0.963 y Score = 0.962 con una relación train/test de 0.33. En la regresión múltiple el MSE = 1785.8, R2 = 0.994 y Score = 0.993. Son diferencias importantes que indican que el modelo de regresión múltiple predice mejor que el de regresión simple, dado que su error MSE es mucho menor y tanto el R2 y score son mayores y cercanos a 1 lo que indicaria que ese modelo tiene mayor capacidad para explicar la variabilidad de los datos.

######**Modelo de regresión simple**
Error Cuadrático Medio (MSE): 10082.17
Coeficiente de Determinación (R^2): 0.963
Score del modelo: 0.962


######**Modelo de regresión múltiple**
Error Cuadrático Medio (MSE): 1785.8 Coeficiente de Determinación (R^2): 0.994 Score del modelo: 0.993

#Regresión múltiple con PCA

In [33]:
from sklearn.decomposition import PCA

# Instancio PCA
n_components = 1  # Número de componentes principales deseado
pca = PCA(n_components=n_components)

# Ajusto y transformo los datos de entrenamiento y test
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Entreno el modelo con los datos transformados por pca
modelo_pca = LinearRegression()
modelo_pca.fit(X_train_pca, y_train)

# Evalúo el modelo
y_pred_pca = modelo_pca.predict(X_test_pca)
mse_pca = mean_squared_error(y_test, y_pred_pca)
r2_pca = r2_score(y_test, y_pred_pca)
_score_pca = modelo_pca.score(X_test_pca, y_test)

# Imprimo
print(f'Error Cuadrático Medio (MSE): {mse_pca.round(3)}')
print(f'Coeficiente de Determinación (R^2): {r2_pca.round(3)}')
print(f'Score del modelo: {_score_pca.round(3)}')
print(f'Modelo con {X_train_pca.shape[1]} features.')


Error Cuadrático Medio (MSE): 57152.347
Coeficiente de Determinación (R^2): 0.793
Score del modelo: 0.793
Modelo con 1 features.


######Las métricas de la regresión multiple con un total de 89 features había dado MSE = 1785.8, R2 = 0.994 y Score = 0.993 luego de aplicar la reducción dimensional a 3 componentes usando PCA los valores de este nuevo modelo son: MSE = 1836.17, R2 = 0.993 y Score = 0.993 con una relación train/test de 0.33. Como puede observarse los valores son muy similares en ambos modelos, por lo que tendrían la misma capacidad predictiva, sin embargo el modelo luego de aplicar PCA y generar tres componentes artificiales como reemplazo de las 89 features del modelo sin PCA, generaría una importante reducción de las dimensiones (89 a 3) con la enorme ventaja de la velocidad y eficiencia que podría manifestarse si el dataset tuviera no miles sino millones de observaciones y no cien sino miles de dimensiones. Y ello sin desmedro de la capacidad predictiva.

######Para degradar un poco los resultados usaré menos componentes en el PCA a efectos de probar si con menos componentes pero usando validación cruzada es posible compensar la perdida de performance del modelo.

######**Modelo PCA con 3 features:**
Error Cuadrático Medio (MSE): 1836.187
Coeficiente de Determinación (R^2): 0.993
Score del modelo: 0.993


######**Modelo PCA con 2 features:**
Error Cuadrático Medio (MSE): 11714.41
Coeficiente de Determinación (R^2): 0.958
Score del modelo: 0.958

######**Modelo PCA con 1 feature:**
Error Cuadrático Medio (MSE): 57152.347
Coeficiente de Determinación (R^2): 0.793
Score del modelo: 0.793

#Regresión múltiple con Feature selection (no usaré este método en este desafío complementario)

In [32]:
# Entreno un arbol de decisión con el objetivo que elija las características de mayor importancia
from sklearn.tree import DecisionTreeRegressor
arbol = DecisionTreeRegressor()
arbol.fit(X_train, y_train)

# Selecciono  características cuya importancia supera la mediana
from sklearn.feature_selection import SelectFromModel
selector = SelectFromModel(arbol, threshold='median')
selector.fit(X_train, y_train)

# Transformo los datos de entrenamiento y prueba quedando las características seleccionadas
X_train_selecionadas = selector.transform(X_train)
X_test_selecionadas = selector.transform(X_test)

# Entreno el modelo con las características seleccionadas según árbol
modelo_selecionadas = LinearRegression()
modelo_selecionadas.fit(X_train_selecionadas, y_train)

# Evalúo el modelo
y_pred_selected = modelo_selecionadas.predict(X_test_selecionadas)
mse_selected = mean_squared_error(y_test, y_pred_selected)
r2_selected = r2_score(y_test, y_pred_selected)
_score_selected = modelo_selecionadas.score(X_test_selecionadas, y_test)

# Imprimo
print(f'Error Cuadrático Medio (MSE): {mse_selected.round(3)}')
print(f'Coeficiente de Determinación (R^2): {r2_selected.round(3)}')
print(f'Score del modelo: {_score_selected.round(3)}')
print(f'Modelo con {X_train_selecionadas.shape[1]} features.')


Error Cuadrático Medio (MSE): 1783.091
Coeficiente de Determinación (R^2): 0.994
Score del modelo: 0.994
Modelo con 45 features.


######Las métricas de la regresión multiple con un total de 89 features había dado MSE = 1785.8, R2 = 0.994 y Score = 0.993 luego de aplicar la reducción dimensional a 3 componentes usando PCA los valores de este nuevo modelo son: MSE = 1836.17, R2 = 0.993 y Score = 0.993 con una relación train/test de 0.33 y usando Feature Selection en lugar de PCA, es decir usando un algoritmo que elije mas que generar nuevas caracteristicas, los valores son MSE = 1783.02, R2 = 0.994 y Score = 0.994. Como puede observarse los valores son muy similares en los tres modelos, en realidad mejores en el feature selection pero seleccionó 45 características entre las 89, donde el PCA habia gennerado solo  tres características artificiales y con buenas o similares métricas por lo que tendrían la misma capacidad predictiva, sin embargo el modelo luego de aplicar PCA y generar tres componentes artificiales como reemplazo de las 89 features del modelo sin PCA, generaría una importante reducción de las dimensiones (89 a 3).

#CONCLUSIONES PCA Y FEATURE SELECTION EN REGRESIÓN LINEAL

######Se probaron cuatro modelos: regresión simple (1 caracteristica), regresión múltiple con codificación vía get_dummies (89 características) y regresión múltiple aplicando antes PCA (solo 3 características) y regresión múltiple aplicando antes Feature Selection (eligió 45 características). En cuanto a la evaluación de MSE, R2 y _score resultaron mejores en este orden: Selection, PCA, Multidimensional y por último Simple. Sin embargo es de destacar que la PCA podría ser la mejor opción para un dataset mucho mas grande que el presentado aquí, con millones de observaciones y miles de columnas originales.

#**PARTE 2. MEJORAS DEL MODELO CON VALIDACIÓN CRUZADA K-FOLD**

#### Elijo uno de los modelos anteriores (en mi caso el problema es Regresion múltiple), por ejemplo el modelo correspondiente a PCA con 1 componente principal (para quedarme con el de peor resultado e intentaré mejorarlo con las técnicas que siguen), buscaré sus mejores hiper parámetros con GridSearchCV, teniendo ese nuevo modelo "mejor_modelo" lo entrenaré y evaluaré. Pero para entrenar puedo usar un split manual del dataset como veniamos haciendo (validacion simple) o un k-Split (Validación cruzada). Luego entreno con esos datos divididos en Train/Test según uno de esos métodos y evalúo si el desempeño del modelo mejoró.

In [66]:
# TENGO UN PROBLEMA DE REGRESIÓN LINEAL MÚLTIPLE

X = data.drop(columns = ['Unit Price'])   # todas menos 'Unit Price'
y = data['Unit Price']                    # Variable a predecir

#HAGO UNA VALIDACIÓN SIMPLE DIVIDIENDO EL DATASET EN TRAIN 70% / TEST 30%

# Separo el conjunto en datos para Train y para Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.98, random_state=42)

# Elijo, selecciono, creo o instancio el modelo de regresión lineal
modelo = LinearRegression()

# Ajusto o entreno el modelo
# mostrándole y_train correspondiente a cada X_train
modelo.fit(X_train, y_train)

# Calculo predicciones con el modelo ya entrenado y con las X de test
y_pred = modelo.predict(X_test)

# Evalúo el modelo comparando su predicción "y_pred" con lo que debería ser "y_test"
# uso dos métricas aplicables en regresión: error cuadrático medio y R^2
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
_score = modelo.score(X,y)

# Imprimo
print(f'Error Cuadrático Medio (MSE): {mse.round(3)}')
print(f'Coeficiente de Determinación (R^2): {r2.round(3)}')
print(f'Score del modelo: {_score.round(3)}')
print(f'Modelo sin pca ni selección de características con {X_train.shape[1]} features.')

#SIGO HACIENDO UNA VALIDACIÓN SIMPLE USANDO UNA DIVISION 70/30 TRAIN/TEST
#PERO ANTES AL DATASET LO REDUZCO DIMENSIONALMENTE CREANDO CARACTERISTICAS PRINCIPALES ARTIFICIALES
#CON LA TÉCNICA DE PCA USANDO INTENCIONALMENTE SOLO 1 CARACTERISTICA PRINCIPAL
from sklearn.decomposition import PCA

# Instancio PCA
n_components = 1  # Número de componentes principales deseado
pca = PCA(n_components=n_components)

# Ajusto y transformo los datos de entrenamiento y test
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Entreno el modelo con los datos transformados por pca
modelo_pca = LinearRegression()
modelo_pca.fit(X_train_pca, y_train)

# Evalúo el modelo
y_pred_pca = modelo_pca.predict(X_test_pca)
mse_pca = mean_squared_error(y_test, y_pred_pca)
r2_pca = r2_score(y_test, y_pred_pca)
_score_pca = modelo_pca.score(X_test_pca, y_test)

# Imprimo
print('')
print(f'Error Cuadrático Medio (MSE): {mse_pca.round(3)}')
print(f'Coeficiente de Determinación (R^2): {r2_pca.round(3)}')
print(f'Score del modelo: {_score_pca.round(3)}')
print(f'Modelo con pca {X_train_pca.shape[1]} features.')

Error Cuadrático Medio (MSE): 2018.101
Coeficiente de Determinación (R^2): 0.993
Score del modelo: 0.993
Modelo sin pca ni selección de características con 89 features.

Error Cuadrático Medio (MSE): 58358.86
Coeficiente de Determinación (R^2): 0.789
Score del modelo: 0.789
Modelo con pca 1 features.


In [67]:
#AHORA QUIERO USAR EL MODELO CON ESE DATASET REDUCIDO POR PCA PERO HACERLE UNA
#VALIDACIÓN CRUZADA CON K-FOLD CROSS VALIDATION CON K=5 Y K=10 Y EN AMBOS CASOS
#OBTENER LAS MISMAS METRICAS QUE ANTES PARA COMPARAR:

from sklearn.model_selection import cross_val_score

modelo_cross = modelo_pca

# Validación cruzada con K = 5 = kinf
kinf = 3
scores_5_fold = cross_val_score(modelo_cross, X_train_pca, y_train, cv=kinf, scoring='neg_mean_squared_error')
mse_cv_5_fold = -scores_5_fold.mean()  # La media de los errores cuadráticos medios negativos

# Validación cruzada con K = 10 = ksup
ksup = 10
scores_10_fold = cross_val_score(modelo_cross, X_train_pca, y_train, cv=ksup, scoring='neg_mean_squared_error')
mse_cv_10_fold = -scores_10_fold.mean()  # La media de los errores cuadráticos medios negativos

# Calcular R^2 y Score con PCA
r2_cv_5_fold = cross_val_score(modelo_cross, X_train_pca, y_train, cv=kinf, scoring='r2').mean()
score_cv_5_fold = cross_val_score(modelo_cross, X_train_pca, y_train, cv=kinf, scoring='r2').mean()
r2_cv_10_fold = cross_val_score(modelo_cross, X_train_pca, y_train, cv=ksup, scoring='r2').mean()
score_cv_10_fold = cross_val_score(modelo_cross, X_train_pca, y_train, cv=ksup, scoring='r2').mean()

# Imprimir los resultados
print(f'k-fold cross validation k={kinf}')
print(f'Error Cuadrático Medio (MSE): {mse_cv_5_fold.round(3)}')
print(f'Coeficiente de Determinación (R^2): {r2_cv_5_fold.round(3)}')
print(f'Score del modelo: {score_cv_5_fold.round(3)}')

print('')
print(f'k-fold cross validation k={ksup}')
print(f'Error Cuadrático Medio (MSE): {mse_cv_10_fold.round(3)}')
print(f'Coeficiente de Determinación (R^2): {r2_cv_10_fold.round(3)}')
print(f'Score del modelo: {score_cv_10_fold.round(3)}')


k-fold cross validation k=3
Error Cuadrático Medio (MSE): 59635.155
Coeficiente de Determinación (R^2): 0.778
Score del modelo: 0.778

k-fold cross validation k=10
Error Cuadrático Medio (MSE): 60077.395
Coeficiente de Determinación (R^2): 0.766
Score del modelo: 0.766


# Conclusiones mejoras de modelos con K-fold cross validation

######Usando validación cruzada de tipo k-fold con k=5 y k=10 y sobre un modelo al que apliqué PCA con componentes n=1, n=2 y n=3 (e incluso n=80) encontré la mayoría de los resultados de evaluación del modelo exactamente iguales, lo cual no me permite sacar una conclusión y me deja la duda de cual es el motivo de que no haya diferencias ante modelos tan diferentes. Incluso probé con k=2 a K=100 y me dió lo mismo. Decidí probar con un modelo entrenado con un split tipico 70/30 train/test y el resultado fué similar sin diferencias. Finalmente ya resignado decidí probar un modelo entrenado con un split extremo y nada recomendado 2% train/ 98% test y recién en ese caso pude encontrar las diferencias marcadas mas arriba ejemplo score 0.778 en el k=5 y score 0.766 en el k=10. Consulto con los que saben, mi tutora en este caso, para ver que puede estar sucediendo.

# **FIN DEL DESAFÍO**